
# UAS Machine Learning — Online Retail II (Dataset 2)

**Nama:** _(Sekar Manopo)_  
**NIM:** 227006416134  
**Kelas/Absen:** _(8)_

---

## Tujuan
Melakukan **segmentasi pelanggan** menggunakan fitur **RFM (Recency, Frequency, Monetary)** dengan algoritma **K-Means Clustering** untuk mengelompokkan pelanggan berdasarkan perilaku belanja, sehingga dapat digunakan untuk strategi pemasaran yang lebih tepat sasaran.

**Alasan pemilihan algoritma:**
- **K-Means** cocok untuk pengelompokan data numerik seperti RFM.
- Cepat, sederhana, dan hasilnya mudah diinterpretasikan untuk profiling pelanggan.
- RFM telah terbukti relevan pada data transaksi ritel untuk mengetahui pelanggan bernilai tinggi, loyal, atau berisiko churn.

> Catatan aturan soal: NIM **genap** → memilih dataset **genap**. Di sini digunakan **Dataset 2 — Online Retail II** dari UCI.



## 1) Persiapan Lingkungan
Jalankan sel berikut pada **Google Colab**.


In [ ]:

# Jika berjalan di Colab, instal paket yang diperlukan
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False

if IN_COLAB:
    !pip -q install mlxtend openpyxl kmodes



## 2) Unduh & Muat Dataset
Dataset: **Online Retail II** (UCI Machine Learning Repository). Mengandung transaksi 2009–2011 dari UK-based online retail. Kita gunakan file Excel resmi dari UCI.


In [ ]:

import pandas as pd
import numpy as np

# URL sumber resmi UCI (bisa berubah; gunakan alternatif mirror bila perlu)
url_2009_2010 = "https://archive.ics.uci.edu/static/public/502/online+retail+ii.zip"

# Fungsi pembaca: unduh ZIP lalu baca dua file excel di dalamnya
import io, zipfile, requests

def load_online_retail_ii():
    # Unduh zip
    r = requests.get(url_2009_2010, timeout=60)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    # List file untuk ditampilkan
    zfile_names = z.namelist()
    print("Files in zip:", zfile_names)
    # Biasanya berisi dua file Excel: 'online_retail_II.xlsx' dengan dua sheet, atau dua file .xlsx terpisah
    # Coba baca cerdas: jika ada .xlsx, baca semua sheet
    xlsx_names = [f for f in zfile_names if f.lower().endswith(".xlsx")]
    if len(xlsx_names) == 0:
        raise RuntimeError("Tidak menemukan file .xlsx di ZIP UCI. Lihat isi daftar di atas dan sesuaikan loader.")
    dfs = []
    for name in xlsx_names:
        with z.open(name) as f:
            xl = pd.ExcelFile(f)
            for sheet in xl.sheet_names:
                df = xl.parse(sheet_name=sheet)
                df["__source_sheet"] = sheet
                dfs.append(df)
    data = pd.concat(dfs, ignore_index=True)
    return data

raw = load_online_retail_ii()
print("Shape:", raw.shape)
raw.head()



## 3) Pembersihan Data
- Hilangkan transaksi **cancel** (Invoice str diawali 'C' atau Quantity negatif).  
- Hanya ambil **Quantity > 0** dan **UnitPrice > 0**.  
- Buang **missing Customer ID**.  
- Buat kolom **TotalPrice = Quantity × UnitPrice**.  
- (Opsional) Fokus pada **United Kingdom** untuk konsistensi.


In [ ]:

# Salin data
df = raw.copy()

# Normalisasi nama kolom agar konsisten
df.columns = [c.strip().replace(' ', '') for c in df.columns]

# Identifikasi kolom-kolom penting secara fleksibel
col_map = {}
for c in df.columns:
    lc = c.lower()
    if lc.startswith("invoice"):
        col_map["Invoice"] = c
    elif "stockcode" in lc:
        col_map["StockCode"] = c
    elif "description" in lc:
        col_map["Description"] = c
    elif "quantity" in lc:
        col_map["Quantity"] = c
    elif "invoicedate" in lc:
        col_map["InvoiceDate"] = c
    elif "unitprice" in lc:
        col_map["UnitPrice"] = c
    elif "customer" in lc and "id" in lc:
        col_map["CustomerID"] = c
    elif "country" in lc:
        col_map["Country"] = c

# Konversi tipe
df[col_map["InvoiceDate"]] = pd.to_datetime(df[col_map["InvoiceDate"]], errors="coerce")
df[col_map["Quantity"]] = pd.to_numeric(df[col_map["Quantity"]], errors="coerce")
df[col_map["UnitPrice"]] = pd.to_numeric(df[col_map["UnitPrice"]], errors="coerce")

# Buang cancel & nilai tak valid
mask_cancel = df[col_map["Invoice"]].astype(str).str.startswith("C", na=False)
df = df[~mask_cancel]
df = df[(df[col_map["Quantity"]] > 0) & (df[col_map["UnitPrice"]] > 0)]
df = df.dropna(subset=[col_map["CustomerID"], col_map["InvoiceDate"]])

# Total price
df["TotalPrice"] = df[col_map["Quantity"]] * df[col_map["UnitPrice"]]

# (Opsional) fokus UK jika kolom Country tersedia
if "Country" in col_map:
    df = df[df[col_map["Country"]] == "United Kingdom"]

df.shape, df.head()



## 4) Feature Engineering: RFM
- **Recency:** Selisih hari dari transaksi terakhir pelanggan ke tanggal acuan.  
- **Frequency:** Jumlah invoice unik per pelanggan.  
- **Monetary:** Total belanja (TotalPrice) per pelanggan.


In [ ]:

ref_date = df[col_map["InvoiceDate"]].max() + pd.Timedelta(days=1)
rfm = (
    df.groupby(df[col_map["CustomerID"]]).agg(
        Recency=(col_map["InvoiceDate"], lambda x: (ref_date - x.max()).days),
        Frequency=(col_map["Invoice"], pd.Series.nunique),
        Monetary=("TotalPrice", "sum"),
    )
    .reset_index()
    .rename(columns={col_map["CustomerID"]: "CustomerID"})
)

print("Jumlah pelanggan:", rfm.shape[0])
rfm.describe()



## 5) Standarisasi & Pemilihan **k** (Elbow & Silhouette)


In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

X = rfm[["Recency", "Frequency", "Monetary"]].copy()
X["Monetary"] = X["Monetary"].clip(upper=X["Monetary"].quantile(0.99))  # cap outlier ringan
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

# Uji beberapa k
ks = list(range(2, 9))
inertias, silhouettes = [], []
for k in ks:
    km = KMeans(n_clusters=k, n_init=20, random_state=42)
    km.fit(Xs)
    inertias.append(km.inertia_)
    silhouettes.append(silhouette_score(Xs, km.labels_))

plt.figure()
plt.plot(ks, inertias, marker="o")
plt.title("Elbow Curve (KMeans)")
plt.xlabel("k")
plt.ylabel("Inertia")
plt.show()

plt.figure()
plt.plot(ks, silhouettes, marker="o")
plt.title("Silhouette Score vs k")
plt.xlabel("k")
plt.ylabel("Silhouette")
plt.show()



## 6) Latih K-Means & Profil Klaster


In [ ]:

best_k = 4  # Sesuaikan berdasarkan plot elbow & silhouette
km = KMeans(n_clusters=best_k, n_init=50, random_state=42)
labels = km.fit_predict(Xs)
rfm["Cluster"] = labels

# Profil klaster (median agar robust)
profile = rfm.groupby("Cluster")[["Recency", "Frequency", "Monetary"]].median().sort_index()
display(profile)

# Contoh interpretasi singkat per klaster
def label_cluster(row, profile):
    # Aturan sederhana berdasarkan perbandingan median
    r_med = profile["Recency"].median()
    f_med = profile["Frequency"].median()
    m_med = profile["Monetary"].median()
    if row["Recency"] <= r_med and row["Frequency"] >= f_med and row["Monetary"] >= m_med:
        return "VIP / Loyal High-Value"
    if row["Recency"] > r_med and row["Frequency"] <= f_med:
        return "At-Risk / Churn"
    if row["Frequency"] > f_med and row["Monetary"] < m_med:
        return "Frequent Low-Spender"
    return "Mid-Value"

rfm["Segment"] = rfm.apply(label_cluster, axis=1, profile=profile)
rfm.head()



## 7) Visualisasi 2D (PCA)


In [ ]:

from sklearn.decomposition import PCA

pca = PCA(n_components=2, random_state=42)
coords = pca.fit_transform(Xs)
plot_df = pd.DataFrame(coords, columns=["PC1", "PC2"])
plot_df["Cluster"] = rfm["Cluster"]

import matplotlib.pyplot as plt
plt.figure()
for c in sorted(plot_df["Cluster"].unique()):
    tmp = plot_df[plot_df["Cluster"] == c]
    plt.scatter(tmp["PC1"], tmp["PC2"], label=f"Cluster {c}", s=10)
plt.legend()
plt.title("PCA Scatter by Cluster")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.show()



## 8) (Opsional) Association Rules / Market Basket
Menemukan **produk yang sering dibeli bersama** (Apriori). Bagian ini bisa dijalankan bila diperlukan.


In [ ]:

# Contoh pipeline ringkas (bisa memerlukan RAM besar; batasi pada 50k invoice untuk demo)
from mlxtend.frequent_patterns import apriori, association_rules

# Persiapan data basket (Invoice × StockCode) -> one-hot
use = df.copy()
# Ambil subset agar lebih ringan
invoice_counts = use.groupby(col_map["Invoice"]).size().sort_values(ascending=False)
top_invoices = set(invoice_counts.head(50000).index)  # batasi
use = use[use[col_map["Invoice"]].isin(top_invoices)]

basket = (
    use.groupby([col_map["Invoice"], col_map["StockCode"]])["Quantity"]
    .sum().unstack().fillna(0).astype(int)
)
basket = basket.applymap(lambda x: 1 if x > 0 else 0)

freq_items = apriori(basket, min_support=0.002, use_colnames=True)
rules = association_rules(freq_items, metric="lift", min_threshold=1.0)
rules = rules.sort_values(by="lift", ascending=False)
rules.head(10)



## 9) Simpulan Singkat
- **K-Means** berhasil mengelompokkan pelanggan menjadi beberapa segmen yang bermakna secara bisnis (mis. VIP, At-Risk, dsb.).  
- Hasil **profil klaster** dapat digunakan untuk strategi: retensi pelanggan berisiko, program loyalitas untuk segmen bernilai tinggi, dan upselling untuk segmen potensial.  
- (Opsional) **Association rules** memberikan wawasan pasangan produk untuk cross-selling.

> Simpan notebook dan unggah ke GitHub sesuai instruksi dosen.
